# Prerequisites
Install the `OpenPAI` sdk from `github` and specify your cluster information in `~/openpai.json` (or somewhere you like). (Refer to for [0-install-sdk-specify-openpai-cluster.ipynb]() more details.)

In [ ]:
! opai cluster --name

# Prepare your script locally
We prepare an example to show how reuse the same code in local environment and in `OpenPAI` node.

In [ ]:
%%writefile test.py

import os
import argparse
from openpaisdk.core import in_job_container

def say_hi(name):
    if in_job_container():
        print('Hello, {}! I am in a container: {}'.format(name, os.environ['PAI_CONTAINER_ID']))
    else:
        print('Hello, {}! I am from local script: {}'.format(name, __file__))

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('name', help='your name')
    args = parser.parse_args()
    say_hi(args.name)

In [ ]:
! python test.py Alice

# Submit to `OpenPAI`
Now we will submit above command to `OpenPAI` through the `opai submit` tool. 

In [ ]:
! opai submit -h

Since `OpenPAI` doesnot allow duplicated job name, you must define a unique name with `-j <job-name>`. If not defined, we use unique string as the job name.

In [ ]:
! opai submit -i jupyter/minimal-notebook python test.py Alice

<font color='red'>_Note: you may need to specify `-a <cluster-alias>` if more than one cluster defined_</font>

By clicking the printed job link, you may find below line in the `STDOUT`

```
Hello, Alice! I am in a container: container_e26_1556418566879_0058_01_000002
```
<font color=red>_The auto fetch of command output is under development_</font>